# Foursquare dataset next-POI Recommendation System

First off we import all the necessary libraries:

In [1]:
 %pip install lightning geohash2 wandb polars==0.20.25

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 820.8 kB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.2/27.2 MB 47.3 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 63.0 MB/s eta 0:00:00
  Created wheel for geohash2: filename=geohash2-1.1-py3-none-any.whl size=15543 sha256=988e78ca35b8d3c0519dc4b67fac90664121e46099796a134317dcab2234d07f
  Stored in directory: /root/.cache/pip/wheels/c0/21/8d/fe65503f4f439aef35193e5ec10a14adc945e20ff87eb35895
Successfully built geohash2
  Attempting uninstall: polars
    Found existing installation: polars 0.20.21
    Uninstalling polars-0.20.21:
      Successfully uninstalled polars-0.20.21
Note: you may need to restart the kernel to use updated packages.


In [2]:
# from google.colab import drive

 #drive.mount("/content/drive")

In [3]:
import polars as rs
import torch
import torch.nn as nn
import torch.nn.functional as F
import lightning as pl
import lightning.pytorch as torchpl
from tqdm import tqdm
import numpy as np
from sklearn.preprocessing import LabelEncoder
from dataclasses import dataclass
import wandb

In [4]:
import os

# define WANDB_NOTEBOOK_NAME
os.environ["WANDB_NOTEBOOK_NAME"] = "train.ipynb"

In [5]:
import gc

gc.collect()
# clean CUDA memory
torch.cuda.empty_cache()

# sometimes jupyter notebook does not release memory, we leave this here so a run-all
# can *sometimes* fix leaks

Next, we load the data, we utilize `polars` since it is much more efficient than `pandas` and can handle large datasets with ease.

In [6]:
columns = ["user", "poi", "date", "TZ"]
data = rs.read_csv(
    "/kaggle/input/data-poi/dataset_TIST2015_Checkins/dataset_TIST2015_Checkins.txt",
    has_header=False,
    low_memory=True,
    separator="\t",
)
data.columns = columns

In [7]:
data

user,poi,date,TZ
i64,str,str,i64
50756,"""4f5e3a72e4b053fd6a4313f6""","""Tue Apr 03 18:00:06 +0000 2012""",240
190571,"""4b4b87b5f964a5204a9f26e3""","""Tue Apr 03 18:00:07 +0000 2012""",180
221021,"""4a85b1b3f964a520eefe1fe3""","""Tue Apr 03 18:00:08 +0000 2012""",-240
66981,"""4b4606f2f964a520751426e3""","""Tue Apr 03 18:00:08 +0000 2012""",-300
21010,"""4c2b4e8a9a559c74832f0de2""","""Tue Apr 03 18:00:09 +0000 2012""",240
…,…,…,…
16349,"""4c957755c8a1bfb7e89024f3""","""Mon Sep 16 23:24:11 +0000 2013""",-240
256757,"""4c8bbb6d9ef0224bd2d6667b""","""Mon Sep 16 23:24:13 +0000 2013""",-180
66425,"""513e82a5e4b0ed4f0f3bcf2d""","""Mon Sep 16 23:24:14 +0000 2013""",-180


Differently from what suggested by the professor, we utilize the full TIST2015 dataset, which has a far greater scale compared to the reduced NY one. However, by following the pruning steps detailed in the paper (http://dx.doi.org/10.1145/3477495.3531989, section 5.1), we obtain sequences that are much smaller in size, resulting in a dataset that is usable on Google Colab's free tier (as required by the assignment).

In [8]:
data_users = (
    data.lazy()
    .group_by("user")
    .agg(
        [
            rs.col("poi").n_unique().alias("n_pois"),
            rs.col("poi").count().alias("n_checkins"),
            # turn the rest into a list
            rs.col("poi").alias("pois"),
            rs.col("date").alias("dates"),
            rs.col("TZ").alias("TZs"),
        ]
    )
).collect()

In [9]:
data_users.describe()

statistic,user,n_pois,n_checkins,pois,dates,TZs
str,f64,f64,f64,f64,f64,f64
"""count""",266909.0,266909.0,266909.0,266909.0,266909.0,266909.0
"""null_count""",0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",133455.0,56.477459,124.62537,null,null,null
"""std""",77050.135837,45.968603,140.692138,null,null,null
"""min""",1.0,1.0,1.0,null,null,null
"""25%""",66728.0,30.0,61.0,null,null,null
"""50%""",133455.0,49.0,93.0,null,null,null
"""75%""",200182.0,71.0,148.0,null,null,null
"""max""",266909.0,1246.0,5430.0,null,null,null


## Data Preprocessing

In [10]:
data_culled = data_users.filter(
    (rs.col("n_checkins") > 20) & (rs.col("n_checkins") < 50)
).drop_nulls()

Since the original dataset is huge, we delete it and call the python garbage collector to free up memory. We then proceed with the second pruning step (frequency-based pruning) as detailed in the paper.

In [11]:
del data
del data_users

import gc

gc.collect()

0

In [12]:
# print lengths

print(data_culled["pois"].list.len().min(), data_culled["pois"].list.len().max())

21 49


In [13]:
# extract unique elements from each lists in data_culled["pois"]
out = data_culled.with_columns(
    [
        rs.col("pois").list.unique(),
        rs.col("pois").list.unique().list.len().alias("n_unique_pois"),
    ]
)

In [14]:
out

user,n_pois,n_checkins,pois,dates,TZs,n_unique_pois
i64,u32,u32,list[str],list[str],list[i64],u32
260144,26,33,"[""4a74998cf964a520cdde1fe3"", ""51a6ad7d498edf566af1e64e"", … ""4db1822843a1bea42bccaec3""]","[""Sun Aug 05 20:40:30 +0000 2012"", ""Sun Aug 12 18:58:46 +0000 2012"", … ""Sat Jul 27 23:45:54 +0000 2013""]","[-300, -300, … -240]",26
102169,21,37,"[""4f24b28fe4b0eec7fe812f8d"", ""4e3dd5641f6e844231eb8cbb"", … ""4e98bfa1b634d75c183586d5""]","[""Wed Apr 11 22:53:29 +0000 2012"", ""Wed Apr 11 23:09:12 +0000 2012"", … ""Sun Jun 10 04:34:19 +0000 2012""]","[-300, -300, … -300]",21
218599,34,47,"[""4b50812cf964a520672527e3"", ""4debba2422713dd973b4d782"", … ""41059b00f964a520850b1fe3""]","[""Sat Apr 07 12:46:03 +0000 2012"", ""Sat Apr 28 22:41:07 +0000 2012"", … ""Sun Sep 15 02:59:51 +0000 2013""]","[120, -240, … 120]",34
149335,23,25,"[""4d4f75f99ffc236ad4550fa7"", ""4c79232ca868370487670f4d"", … ""50d2ced1e4b0d38a8f6be15d""]","[""Thu Jun 07 00:47:13 +0000 2012"", ""Sat Jun 09 09:08:05 +0000 2012"", … ""Sun Aug 25 08:24:39 +0000 2013""]","[480, 480, … 480]",23
154551,11,48,"[""4b058717f964a520527f22e3"", ""4b16e51cf964a52016bf23e3"", … ""4ccccdf0294aef3be34c85e2""]","[""Mon Apr 09 21:04:09 +0000 2012"", ""Thu Apr 12 22:39:17 +0000 2012"", … ""Sun Jul 14 19:52:38 +0000 2013""]","[-180, -180, … -180]",11
…,…,…,…,…,…,…
75667,19,27,"[""409ad180f964a520eef21ee3"", ""4f552067e4b028a92259bce9"", … ""4a348baef964a520639c1fe3""]","[""Sun Apr 08 23:29:45 +0000 2012"", ""Fri Apr 20 20:39:25 +0000 2012"", … ""Sat Aug 03 20:31:38 +0000 2013""]","[-420, -420, … -420]",19
254236,17,35,"[""4e5a7f82ae6099fc00d5d967"", ""50e089d4e4b0f3400b7e31ff"", … ""4e74f3f0d22d80eb32fce474""]","[""Mon Oct 29 05:37:55 +0000 2012"", ""Wed Oct 31 15:50:56 +0000 2012"", … ""Wed Jul 24 08:45:27 +0000 2013""]","[120, 120, … 180]",17
162845,36,40,"[""4f03eca8d3e35fb2b715f469"", ""4cd4ebd967c7236a1a381077"", … ""4b0587fdf964a52034ab22e3""]","[""Mon Oct 22 10:37:00 +0000 2012"", ""Tue Oct 23 02:01:43 +0000 2012"", … ""Wed May 29 08:23:59 +0000 2013""]","[420, 420, … 420]",36


In [15]:
l = out["pois"][0].to_list()
len(set(l))  # print number of unique POIs in first sequence

26

In [16]:
l2 = data_culled["pois"][0].to_list()
len(l2)  # print sequence length of first user

33

In [17]:
len(set(l2))  # confirm that the two match

26

In [18]:
# run a Polars query to obtain all the frequent POIs, the ones expected to survive the filtering
unique_pois = out["pois"]
frequent_pois = unique_pois.list.explode().value_counts().filter(rs.col("count") >= 10)

In [19]:
frequent_pois

pois,count
str,u32
"""4b59ebdaf964a52002a128e3""",25
"""4ba8b203f964a520a6e739e3""",15
"""4b60500bf964a5200edf29e3""",94
"""4c24f767f1272d7ff6f283c5""",10
"""4b8edb30f964a5201f3b33e3""",101
…,…
"""4b4dab8df964a52084d526e3""",10
"""4bae1656f964a520ad823be3""",12
"""4a106621f964a520ba761fe3""",17


In [20]:
frequent_pois = frequent_pois["pois"]
frequent_pois = set(frequent_pois.to_list())

In [21]:
data_culled

user,n_pois,n_checkins,pois,dates,TZs
i64,u32,u32,list[str],list[str],list[i64]
260144,26,33,"[""4ff9b1eee4b024f55b00c99c"", ""4ff9b1eee4b024f55b00c99c"", … ""500470e68302e49d1a79f80a""]","[""Sun Aug 05 20:40:30 +0000 2012"", ""Sun Aug 12 18:58:46 +0000 2012"", … ""Sat Jul 27 23:45:54 +0000 2013""]","[-300, -300, … -240]"
102169,21,37,"[""4f3cbb9be4b0c30dfedd3e76"", ""4f57c2b6e4b09ba0e2a0e376"", … ""4d8ffdd3788c548144fc5bfd""]","[""Wed Apr 11 22:53:29 +0000 2012"", ""Wed Apr 11 23:09:12 +0000 2012"", … ""Sun Jun 10 04:34:19 +0000 2012""]","[-300, -300, … -300]"
218599,34,47,"[""4debba2422713dd973b4d782"", ""3fd66200f964a520d6f01ee3"", … ""4b7a230bf964a52034242fe3""]","[""Sat Apr 07 12:46:03 +0000 2012"", ""Sat Apr 28 22:41:07 +0000 2012"", … ""Sun Sep 15 02:59:51 +0000 2013""]","[120, -240, … 120]"
149335,23,25,"[""4ca89516d971b1f7d5b915e1"", ""4eec6e418b81bbbf5bf1c6cd"", … ""50377d24e4b05b4c7a1541f8""]","[""Thu Jun 07 00:47:13 +0000 2012"", ""Sat Jun 09 09:08:05 +0000 2012"", … ""Sun Aug 25 08:24:39 +0000 2013""]","[480, 480, … 480]"
154551,11,48,"[""4dc7d76a1f6ef43b8a4e2228"", ""4dc7d76a1f6ef43b8a4e2228"", … ""4ccccdf0294aef3be34c85e2""]","[""Mon Apr 09 21:04:09 +0000 2012"", ""Thu Apr 12 22:39:17 +0000 2012"", … ""Sun Jul 14 19:52:38 +0000 2013""]","[-180, -180, … -180]"
…,…,…,…,…,…
75667,19,27,"[""4abee430f964a520709020e3"", ""4ea836928231a8f6a1c60d83"", … ""4bd2177d046076b055357371""]","[""Sun Apr 08 23:29:45 +0000 2012"", ""Fri Apr 20 20:39:25 +0000 2012"", … ""Sat Aug 03 20:31:38 +0000 2013""]","[-420, -420, … -420]"
254236,17,35,"[""4c1a83c5b306c928abba60b7"", ""4e4e9e726284416669825954"", … ""505af018e4b0a3801ae77843""]","[""Mon Oct 29 05:37:55 +0000 2012"", ""Wed Oct 31 15:50:56 +0000 2012"", … ""Wed Jul 24 08:45:27 +0000 2013""]","[120, 120, … 180]"
162845,36,40,"[""4b2df07cf964a5201bdc24e3"", ""4b347112f964a520852725e3"", … ""4b2df07cf964a5201bdc24e3""]","[""Mon Oct 22 10:37:00 +0000 2012"", ""Tue Oct 23 02:01:43 +0000 2012"", … ""Wed May 29 08:23:59 +0000 2013""]","[420, 420, … 420]"


In [22]:
data_culled = data_culled.with_columns(
    [
        rs.col("pois")
        .list.eval(
            rs.element().is_in(frequent_pois),
        )
        .alias("is_frequent")
    ]
)  # prep mask

In [23]:
final_data = (
    data_culled.lazy()
    .explode(
        [
            "pois",
            "dates",
            "TZs",
            "is_frequent",
        ]
    )
    .group_by("user")
    .agg(
        [
            rs.col("pois").filter(rs.col("is_frequent")).alias("pois"),
            rs.col("dates").filter(rs.col("is_frequent")).alias("dates"),
            rs.col("TZs").filter(rs.col("is_frequent")).alias("TZs"),
            rs.col("pois").filter(rs.col("is_frequent")).n_unique().alias("n_pois"),
            rs.col("pois").filter(rs.col("is_frequent")).count().alias("n_checkins"),
        ]
    )
    .filter(rs.col("n_checkins") > 0)
    .filter(rs.col("n_pois") > 0)
    .collect()
)  # filter out infrequent pois and users with no pois

In [24]:
final_data.describe()

statistic,user,pois,dates,TZs,n_pois,n_checkins
str,f64,f64,f64,f64,f64,f64
"""count""",19862.0,19862.0,19862.0,19862.0,19862.0,19862.0
"""null_count""",0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",156852.822274,null,null,null,6.123452,8.831437
"""std""",76314.892884,null,null,null,4.609024,6.877662
"""min""",49.0,null,null,null,1.0,1.0
"""25%""",95613.0,null,null,null,3.0,4.0
"""50%""",167846.0,null,null,null,5.0,7.0
"""75%""",224576.0,null,null,null,8.0,12.0
"""max""",266909.0,null,null,null,32.0,46.0


At this stage, culling is done, we can appreciate that `polars`'s SQL/functional-style API is different from Pandas, but it is very powerful and efficient.

The next step is geohashing the POIs, that is, we want to convert the latitude-longitude positions of the POIs into a grid-based geohash representation, which will form the basis for our network's embeddings.

In [25]:
import geohash2 as gh

pois = rs.read_csv(
    "/kaggle/input/data-poi/dataset_TIST2015_POIs.txt",
    has_header=False,
    low_memory=True,
    separator="\t",
)
pois.columns = ["poi", "lat", "long", "category", "country"]
pois = pois.drop("category").drop("country")

In [26]:
pois = (
    pois.lazy()
    .filter(rs.col("poi").is_in(frequent_pois))
    .select(
        [
            rs.col("poi"),
            rs.struct(
                [
                    rs.col("lat").cast(rs.Float32),
                    rs.col("long").cast(rs.Float32),
                ]
            )
            .alias("location")
            .map_elements(
                lambda s: gh.encode(s["lat"], s["long"], precision=6),
                return_dtype=rs.String,
            )
            .alias("geohash"),
        ]
    )
    .collect()
)
poi_geo_dict = dict(zip(pois["poi"], pois["geohash"]))

In [27]:
# for each row in final_data, add the geohash of the pois by hitting the poi_geo_dict

final_data = final_data.with_columns(
    [
        rs.col("pois")
        .map_elements(
            lambda s: [poi_geo_dict[s] for s in s],
            return_dtype=rs.List(rs.String),
        )
        .alias("geohashes")
    ]
)

In [28]:
final_data["dates"][79].to_list()  # check out a temporal sequence

['Wed Jun 13 15:42:57 +0000 2012', 'Mon Jul 02 19:21:44 +0000 2012']

In [29]:
final_data["TZs"][79].to_list()  # ... and the corresponding timezones

[180, 180]

The work *might* seem over, however, we still have timezones to account for, we want to normalize everything according to GMT, so we convert the timestamps accordingly.

In [30]:
import datetime


def UTC_to_local(utc, tz):

    date = datetime.datetime.strptime(utc, "%a %b %d %H:%M:%S %z %Y")
    date = date.replace(tzinfo=datetime.timezone.utc)

    # shift by tz offset
    date = date.astimezone(datetime.timezone(datetime.timedelta(minutes=tz)))

    date_s = datetime.datetime.strftime(date, "%Y-%m-%d %H:%M:%S")
    return date_s


def to_UNIX_time(date):
    return datetime.datetime.strptime(
        date, "%Y-%m-%d %H:%M:%S"
    ).timestamp()  # we use UNIX time as a key for sorting the POIs in our polars query

In [31]:
UTC_to_local("Mon May 21 15:53:01 +0000 2012", -420)  # example of usage

'2012-05-21 08:53:01'

In [32]:
final_data = final_data.with_columns(
    [
        rs.struct([rs.col("dates"), rs.col("TZs")])
        .alias("times")
        .map_elements(
            lambda struct: [
                UTC_to_local(date, tz)
                for date, tz in zip(struct["dates"], struct["TZs"])
            ],
            return_dtype=rs.List(rs.String),
        )
    ]
)  # This performs timezone conversion

In [33]:
final_sorted = final_data.select(  # sort the times
    [
        rs.col("user"),
        rs.struct(
            [
                rs.col("pois"),
                rs.col("times"),
            ]
        ).map_elements(
            lambda struct: [
                poi
                for poi, _ in sorted(
                    zip(  # here we sort the POIs struct by UNIX timestamps of the GMT times
                        struct["pois"], [to_UNIX_time(date) for date in struct["times"]]
                    ),
                    key=lambda s: s[1],
                )
            ],
            return_dtype=rs.List(rs.String),
        ),
        rs.struct(
            [
                rs.col("geohashes"),
                rs.col("times"),
            ]
        ).map_elements(
            lambda struct: [
                geo
                for geo, _ in sorted(
                    zip(
                        struct["geohashes"],  # same thing goes on for geohashes
                        [to_UNIX_time(date) for date in struct["times"]],
                    ),
                    key=lambda s: s[1],
                )
            ],
            return_dtype=rs.List(rs.String),
        ),
        rs.col("times")
        .map_elements(
            lambda dates: sorted(dates, key=to_UNIX_time),
            return_dtype=rs.List(rs.String),
        )
        .alias("times_sorted"),
        rs.col("n_checkins"),
    ]
)

# P.S, admittedly, it would have been more efficient to encode the geohashes *after* sorting the POIs,
# so that we could save on the sorting of the geohashes. Tough luck, you can't win 'em all.

In [34]:
final_sorted

user,pois,geohashes,times_sorted,n_checkins
i64,list[str],list[str],list[str],u32
25337,"[""4b835901f964a520620331e3"", ""4bebe276ca97d13a65ae6c77""]","[""qqw7xm"", ""qqwe9n""]","[""2012-04-28 21:56:36"", ""2012-04-29 11:32:15""]",2
64712,"[""4c14b6b282a3c9b629abfdf8"", ""4ba1a917f964a5207ec637e3"", … ""4b83bb61f964a5208a0e31e3""]","[""qqu88w"", ""qqu82r"", … ""qqu88w""]","[""2012-04-11 12:40:13"", ""2012-04-11 15:49:37"", … ""2013-07-29 13:35:12""]",9
249372,"[""4d06f54d7d9ba35df0ee7223"", ""4b0587a6f964a5203e9e22e3"", … ""4b0587a6f964a5203e9e22e3""]","[""xn76ev"", ""xn76ur"", … ""xn76ur""]","[""2012-07-05 10:14:26"", ""2012-07-05 19:12:02"", … ""2013-08-30 19:26:49""]",14
67162,"[""42829c80f964a5206a221fe3"", ""42829c80f964a5206a221fe3"", … ""40dcbc80f964a52081011fe3""]","[""dr5rud"", ""dr5rud"", … ""dp3wtk""]","[""2012-04-27 08:49:22"", ""2012-04-27 18:38:13"", … ""2013-08-22 16:15:42""]",10
53443,"[""4c1781ecc2dfc9287e17a869"", ""4b5bdb3df964a520131a29e3"", ""4b8d9c33f964a520140433e3""]","[""qqy822"", ""qqu897"", ""qqgvph""]","[""2012-04-24 12:24:32"", ""2012-06-27 13:42:02"", ""2013-01-10 12:53:23""]",3
…,…,…,…,…
263415,"[""4bb3cd9fa32876b06d3d02fe"", ""4c9930f3775abfb73cd54267"", … ""4c9930f3775abfb73cd54267""]","[""6kqj5c"", ""66tmfd"", … ""66tmfd""]","[""2012-04-21 17:04:57"", ""2012-05-25 23:13:43"", … ""2013-01-27 18:28:51""]",6
241904,"[""4a19d1edf964a5205a7a1fe3"", ""40afe980f964a5203bf31ee3"", … ""4b992b04f964a520726635e3""]","[""dr5rus"", ""dr5ru7"", … ""dr5rkw""]","[""2012-04-11 14:04:26"", ""2012-04-21 19:41:34"", … ""2013-09-07 11:24:49""]",11
95755,"[""4b058853f964a5201bbf22e3"", ""4b7a8ad6f964a52075302fe3"", … ""4b05884ff964a520adbd22e3""]","[""u0yhep"", ""u0yhep"", … ""u0yjj1""]","[""2012-05-27 11:27:06"", ""2012-05-27 11:51:12"", … ""2013-02-25 16:29:14""]",13


we now need to obtain a dataframe containing: each POI, it's geohash, and a set of all the check-ins it appears in
this is just one `polars` query away!

In [35]:
pois_checkins = final_sorted.explode(["pois", "geohashes"]).drop("n_checkins")

pois_checkins = (
    pois_checkins.with_columns(
        [
            rs.col("geohashes").map_elements(lambda s: s[:4], rs.String).alias("g4"),
        ]
    )
    .drop("geohashes")
    .group_by(["pois", "g4"])
    .agg([rs.col("times_sorted").flatten().alias("checkin_times")])
)

In [36]:
pois_checkins  # with this we can *efficiently* build our POI-POI spatial-temporal graphs

pois,g4,checkin_times
str,str,list[str]
"""4d160cd685fc6dcb5649a44e""","""w0zf""","[""2013-04-26 09:53:53"", ""2013-04-28 16:16:56"", … ""2013-04-15 17:14:50""]"
"""4adcdb27f964a520dd6121e3""","""ud9z""","[""2012-04-14 15:16:53"", ""2012-07-02 09:46:54"", … ""2013-04-03 12:06:11""]"
"""4f7192fee4b022ab2187553e""","""swtc""","[""2012-12-22 16:28:39"", ""2012-12-27 18:34:25"", … ""2013-06-05 16:35:33""]"
"""4be574b55254d13a978560e1""","""wdw5""","[""2013-01-28 16:00:56"", ""2013-01-28 16:15:03"", … ""2013-08-05 06:43:31""]"
"""4ba348b7f964a5203d3338e3""","""w4rq""","[""2013-04-18 17:29:05"", ""2013-04-21 18:29:53"", … ""2013-05-01 09:34:14""]"
…,…,…
"""4b30d634f964a520fbfb24e3""","""w4rq""","[""2012-04-12 15:40:26"", ""2012-04-13 15:40:02"", … ""2013-05-29 15:23:59""]"
"""4ba2fdc6f964a520b02638e3""","""9v6k""","[""2012-04-22 15:03:01"", ""2012-11-25 16:59:37"", … ""2013-05-21 18:39:27""]"
"""4cd103d96449a0936af0cdcf""","""sxk9""","[""2012-09-05 13:06:37"", ""2012-09-09 16:17:13"", … ""2012-12-17 04:47:51""]"


In [37]:
def UTC_to_weekslot(utc: str) -> int:
    """UTC_to_weekslot converts a UTC timestamp to a weekslot.

    Parameters
    ----------
    utc : str
        A string representing a UTC timestamp.

    Returns
    -------
    int
        A weekslot in the range [0, 56).
    """

    date = datetime.datetime.strptime(utc, "%Y-%m-%d %H:%M:%S")
    week = date.weekday()
    hour = date.hour

    return week * 8 + hour // 3

Next, we want to encode all of our inputs for our neural networks, this could *probably* be done
with polars magic, but it's too delicate and we prefer classic for-looping.

In [38]:
encoder_dict = {
    "users": LabelEncoder(),
    "pois": LabelEncoder(),
    "g2": LabelEncoder(),
    "g3": LabelEncoder(),
    "g4": LabelEncoder(),
    "g5": LabelEncoder(),
    "g6": LabelEncoder(),
}

encoded_data = {
    "users": [],
    "pois": [],
    "g2": [],
    "g3": [],
    "g4": [],
    "g5": [],
    "g6": [],
}

unique_data = {
    "users": set(),
    "pois": set(),
    "g2": set(),
    "g3": set(),
    "g4": set(),
    "g5": set(),
    "g6": set(),
}

# quick and dirty encoding:
# 1. put every unique symbol in a list
# 2. fit the respective encoder
# 3. transform the lists

for i, row in enumerate(final_sorted.iter_rows()):

    user, pois, geohashes, times_sorted, n_checkins = row

    g2 = [geo[:2] for geo in geohashes]
    g3 = [geo[:3] for geo in geohashes]
    g4 = [geo[:4] for geo in geohashes]
    g5 = [geo[:5] for geo in geohashes]
    g6 = [geo[:6] for geo in geohashes]  # redundant, but I like symmetry

    unique_data["users"].add(user)
    unique_data["pois"].update(pois)
    unique_data["g2"].update(g2)
    unique_data["g3"].update(g3)
    unique_data["g4"].update(g4)
    unique_data["g5"].update(g5)
    unique_data["g6"].update(g6)

for property, enc, data in zip(
    encoder_dict.keys(), encoder_dict.values(), unique_data.values()
):
    enc.fit(list(data))
    encoder_dict[property] = enc

In [39]:
# this could be optimized, right now it takes a while, at least we have a nice progress bar to look at

ds_size = len(final_sorted)

for i, row in tqdm(enumerate(final_sorted.iter_rows()), total=ds_size):

    user, pois, geohashes, times_sorted, n_checkins = row

    g2 = [geo[:2] for geo in geohashes]
    g3 = [geo[:3] for geo in geohashes]
    g4 = [geo[:4] for geo in geohashes]
    g5 = [geo[:5] for geo in geohashes]
    g6 = [geo[:6] for geo in geohashes]

    encoded_data["users"].append(encoder_dict["users"].transform([user])[0])
    encoded_data["pois"].append(encoder_dict["pois"].transform(pois))
    encoded_data["g2"].append(encoder_dict["g2"].transform(g2))
    encoded_data["g3"].append(encoder_dict["g3"].transform(g3))
    encoded_data["g4"].append(encoder_dict["g4"].transform(g4))
    encoded_data["g5"].append(encoder_dict["g5"].transform(g5))
    encoded_data["g6"].append(encoder_dict["g6"].transform(g6))

    # sum 1 to all values to avoid 0s
    encoded_data["users"][-1] += 1
    encoded_data["pois"][-1] += 1
    encoded_data["g2"][-1] += 1
    encoded_data["g3"][-1] += 1
    encoded_data["g4"][-1] += 1
    encoded_data["g5"][-1] += 1
    encoded_data["g6"][-1] += 1

100%|██████████| 19862/19862 [04:32<00:00, 72.80it/s]


In [40]:
# check that we left space for the padding token
min((arr.min() for arr in encoded_data["pois"]))

1

In [41]:
pois_checkins

pois,g4,checkin_times
str,str,list[str]
"""4d160cd685fc6dcb5649a44e""","""w0zf""","[""2013-04-26 09:53:53"", ""2013-04-28 16:16:56"", … ""2013-04-15 17:14:50""]"
"""4adcdb27f964a520dd6121e3""","""ud9z""","[""2012-04-14 15:16:53"", ""2012-07-02 09:46:54"", … ""2013-04-03 12:06:11""]"
"""4f7192fee4b022ab2187553e""","""swtc""","[""2012-12-22 16:28:39"", ""2012-12-27 18:34:25"", … ""2013-06-05 16:35:33""]"
"""4be574b55254d13a978560e1""","""wdw5""","[""2013-01-28 16:00:56"", ""2013-01-28 16:15:03"", … ""2013-08-05 06:43:31""]"
"""4ba348b7f964a5203d3338e3""","""w4rq""","[""2013-04-18 17:29:05"", ""2013-04-21 18:29:53"", … ""2013-05-01 09:34:14""]"
…,…,…
"""4b30d634f964a520fbfb24e3""","""w4rq""","[""2012-04-12 15:40:26"", ""2012-04-13 15:40:02"", … ""2013-05-29 15:23:59""]"
"""4ba2fdc6f964a520b02638e3""","""9v6k""","[""2012-04-22 15:03:01"", ""2012-11-25 16:59:37"", … ""2013-05-21 18:39:27""]"
"""4cd103d96449a0936af0cdcf""","""sxk9""","[""2012-09-05 13:06:37"", ""2012-09-09 16:17:13"", … ""2012-12-17 04:47:51""]"


In [42]:
# we also encode the graph dataframe so we can build the graphs

pois_checkins = (
    pois_checkins.lazy()
    .with_columns(
        [
            rs.col("pois").map_elements(
                lambda s: encoder_dict["pois"].transform([s])[0] + 1, rs.Int64
            ),
            rs.col("g4").map_elements(
                lambda s: encoder_dict["g4"].transform([s])[0] + 1, rs.Int64
            ),  # apply utc_to_weekslot to each timestamp in the list
            rs.col("checkin_times").map_elements(
                lambda s: [UTC_to_weekslot(date) for date in s], rs.List(rs.Int64)
            ),
        ]
    )
    .sort("pois")
    .collect()
)

In [43]:
# add fictitious POI 0 to the graph, with nonexistent geohash and no timeslot, so we get a 0 row and column for the padding token
fake_datapoint = rs.DataFrame(
    {
        "pois": [0],
        "g4": [pois_checkins["g4"].max() + 42],
        "checkin_times": [[43]],
    }
)
# this is a lot of work since polars dataframes are immutable by default, we have to run a query to change the 43 into an empty list
# we NEED the 43 otherwise polars won't infer the datatype of the list

fake_datapoint = fake_datapoint.with_columns(
    [rs.col("checkin_times").map_elements(lambda s: [], rs.List(rs.Int64))]
)

pois_checkins = fake_datapoint.vstack(pois_checkins)

In [44]:
spatial_row = np.array(pois_checkins["g4"].to_list()).reshape(-1, 1)

In [45]:
# outer product using equality
spatial_graph = (spatial_row == spatial_row.T).astype(np.int32)
spatial_graph[0, 0] = (
    0  # the fake g4 is still equal to itself, we suppress this equality
)
spatial_graph = torch.tensor(spatial_graph)

In [46]:
temporal_row = pois_checkins["checkin_times"].to_list()

In [47]:
temporal_graph = np.zeros((spatial_row.shape[0], spatial_row.shape[0]))

In [48]:
temporal_sets = [np.array(list(set(row))) for row in temporal_row]

In [49]:
time_sets = torch.zeros((len(temporal_sets), 56), dtype=torch.int8)

for i, r in enumerate(temporal_row):
    indices = torch.tensor(r, dtype=torch.long)
    time_sets[i, indices] = 1

In [50]:
time_sets.shape

torch.Size([4456, 56])

In [51]:
# AND outer product

intersection = time_sets @ time_sets.T
union = time_sets.unsqueeze(1) | time_sets.unsqueeze(0)
union = union.sum(dim=2)
iou = intersection / union

In [52]:
temporal_graph = iou >= 0.9
# cast to int
temporal_graph = temporal_graph.int()

In [53]:
temporal_graph[0, :].sum()

tensor(0)

We print information about the sparsity of the graphs, we note that
the sparsity of the graphs is similar to that of the paper.

In [54]:
temporal_density = (
    temporal_graph.sum() / (temporal_graph.shape[0] * temporal_graph.shape[1])
).item()
spatial_density = (
    spatial_graph.sum() / (spatial_graph.shape[0] * spatial_graph.shape[1])
).item()

print(f"Temporal sparsity: {(1 - temporal_density) * 100:.2f}%")

print(f"Spatial sparsity: {(1 - spatial_density) * 100:.2f}%")

Temporal sparsity: 97.05%
Spatial sparsity: 96.99%


## Train Test Split

We now generate two dataframes from the `encoded_data` dataframe, one for training and one for testing.

First, we have to drop every sequence that has less than 4 timestamps, as we wouldn't be able to get the minimum of two samples for each of the sets,
we then calculate the 80% of the sequences and split the data accordingly.

In [55]:
len(encoded_data["pois"])

19862

In [56]:
total_data = rs.DataFrame(encoded_data)

In [57]:
total_data = total_data.with_columns(
    [
        rs.col("pois").list.len().alias("length"),
    ]
)

In [58]:
total_data = total_data.with_columns(
    rs.col("length")
    .map_elements(lambda s: int(0.8 * s) - 1, rs.Int64)
    .alias("train_end")
)

In [59]:
# drop sequences that are too short
total_data = total_data.filter(
    (
        rs.col("train_end") >= 1
    )  # at least 2 elements in the training set (1 is the index)
    & (
        rs.col("length") - (rs.col("train_end") + 1) >= 2
    )  # at least 2 elements in the validation set
)
print(total_data["length"].mean())
print(total_data.count())

12.629353233830846
shape: (1, 9)
┌───────┬───────┬───────┬───────┬───┬───────┬───────┬────────┬───────────┐
│ users ┆ pois  ┆ g2    ┆ g3    ┆ … ┆ g5    ┆ g6    ┆ length ┆ train_end │
│ ---   ┆ ---   ┆ ---   ┆ ---   ┆   ┆ ---   ┆ ---   ┆ ---    ┆ ---       │
│ u32   ┆ u32   ┆ u32   ┆ u32   ┆   ┆ u32   ┆ u32   ┆ u32    ┆ u32       │
╞═══════╪═══════╪═══════╪═══════╪═══╪═══════╪═══════╪════════╪═══════════╡
│ 12060 ┆ 12060 ┆ 12060 ┆ 12060 ┆ … ┆ 12060 ┆ 12060 ┆ 12060  ┆ 12060     │
└───────┴───────┴───────┴───────┴───┴───────┴───────┴────────┴───────────┘


In [60]:
total_data.sort("length")  # check out the distribution of sequence lengths

users,pois,g2,g3,g4,g5,g6,length,train_end
i64,list[i64],list[i64],list[i64],list[i64],list[i64],list[i64],u32,i64
5515,"[573, 87, … 443]","[29, 29, … 42]","[54, 58, … 87]","[97, 104, … 148]","[250, 272, … 353]","[457, 498, … 623]",6,3
11509,"[1849, 179, … 119]","[31, 27, … 26]","[63, 51, … 49]","[111, 93, … 88]","[284, 244, … 237]","[524, 450, … 441]",6,3
4877,"[2027, 130, … 443]","[46, 46, … 42]","[115, 115, … 87]","[205, 207, … 148]","[474, 486, … 353]","[843, 874, … 623]",6,3
16287,"[216, 195, … 721]","[33, 26, … 22]","[71, 48, … 39]","[125, 84, … 63]","[311, 226, … 157]","[564, 419, … 286]",6,3
817,"[443, 469, … 397]","[42, 29, … 29]","[87, 59, … 59]","[148, 105, … 105]","[353, 276, … 275]","[623, 508, … 503]",6,3
…,…,…,…,…,…,…,…,…
9814,"[1542, 1542, … 793]","[109, 109, … 109]","[277, 277, … 277]","[538, 538, … 538]","[1319, 1319, … 1322]","[2671, 2671, … 2685]",44,34
18788,"[1561, 1561, … 1255]","[106, 106, … 109]","[266, 266, … 277]","[509, 509, … 541]","[1236, 1236, … 1344]","[2517, 2517, … 2746]",44,34
10822,"[1280, 3441, … 3915]","[68, 68, … 68]","[181, 181, … 181]","[334, 332, … 334]","[807, 795, … 802]","[1637, 1536, … 1585]",45,35


In [61]:
# Check if the shortest sequence is long enough
total_data.sort("length")["pois"][0]

""
i64
573
87
4097
572
487
443


In [62]:
# slice the two dataframes
train_data = total_data.select(
    [
        rs.col("users"),
        rs.struct(
            [
                rs.col("pois"),
                rs.col("g2"),
                rs.col("g3"),
                rs.col("g4"),
                rs.col("g5"),
                rs.col("g6"),
                rs.col("train_end"),
            ]
        )
        .map_elements(
            lambda struct: [
                struct["pois"][: struct["train_end"]],
                struct["g2"][: struct["train_end"]],
                struct["g3"][: struct["train_end"]],
                struct["g4"][: struct["train_end"]],
                struct["g5"][: struct["train_end"]],
                struct["g6"][: struct["train_end"]],
            ],
            return_dtype=rs.List(rs.List(rs.Int64)),
        )
        .alias("sequences"),
    ]
)


test_data = total_data.select(
    [
        rs.col("users"),
        rs.struct(
            [
                rs.col("pois"),
                rs.col("g2"),
                rs.col("g3"),
                rs.col("g4"),
                rs.col("g5"),
                rs.col("g6"),
                rs.col("train_end"),
            ]
        )
        .map_elements(
            lambda struct: [
                struct["pois"][struct["train_end"] :],
                struct["g2"][struct["train_end"] :],
                struct["g3"][struct["train_end"] :],
                struct["g4"][struct["train_end"] :],
                struct["g5"][struct["train_end"] :],
                struct["g6"][struct["train_end"] :],
            ],
            return_dtype=rs.List(rs.List(rs.Int64)),
        )
        .alias("sequences"),
    ]
)

In [63]:
def explode_dict(d):
    """explode_dict Convert packed polars dataframe into a neat python dict

    Parameters
    ----------
    d : Polars.DataFrame
        A polars dataframe with a struct column

    Returns
    -------
    dict
        A python dict with the same structure as the struct column
    """
    ret = {
        "users": d["users"].to_list(),
        "pois": [],
        "g2": [],
        "g3": [],
        "g4": [],
        "g5": [],
        "g6": [],
    }

    columns = ["pois", "g2", "g3", "g4", "g5", "g6"]

    for sample in d["sequences"]:
        pois, g2, g3, g4, g5, g6 = sample
        ret["pois"].append(pois.to_list())
        ret["g2"].append(g2.to_list())
        ret["g3"].append(g3.to_list())
        ret["g4"].append(g4.to_list())
        ret["g5"].append(g5.to_list())
        ret["g6"].append(g6.to_list())

    return ret

In [64]:
encoded_data_train = explode_dict(train_data.to_dict())
encoded_data_test = explode_dict(test_data.to_dict())

## Metrics


The paper utilizes metrics that check if the target is in the top-k recommendations, we implement them here.

In [65]:
class AccuracyAtK(nn.Module):
    def __init__(self, k: int):
        """__init__ initializes the AccuracyAtK module.

        Accuracy@k is the proportion of correct predictions in the top-k elements.

        Parameters
        ----------
        k : int
            The number of top-k elements to consider.

        """
        super().__init__()
        self.k = k

    def forward(self, logits: torch.Tensor, targets: torch.Tensor) -> torch.Tensor:
        """forward computes the accuracy at k between logits and targets.

        Parameters
        ----------
        logits : torch.Tensor
            Class probability, either (B, C) or (B, T, C)
        targets : torch.Tensor
            Ground truth class indices, either (B,) or (B, T)

        Returns
        -------
        torch.Tensor
            The accuracy at k, a scalar-tensor.
        """

        # Gotta have at least one nasty python one-liner, in memory of the old
        # programming lab 1 bachelor course
        return (
            (logits.topk(self.k, dim=-1)[1] == targets.unsqueeze(-1))
            .any(dim=-1)
            .float()
            .mean()
        )
class AccuracyAt1(nn.Module):
      def __init__(self):
          super().__init__()

      def forward(self, logits: torch.Tensor, targets: torch.Tensor) -> torch.Tensor:
            predicted = logits.argmax(dim=-1)
            correct = (predicted == targets).float()
            accuracy = correct.mean()
            return accuracy

class MeanReciprocalRank(nn.Module):

    def __init__(self):
        """__init__ initializes the MeanReciprocalRank module.

        Mean reciprocal rank is the average of the reciprocal ranks of the top-k elements.

        """
        super().__init__()

    def forward(self, logits: torch.Tensor, targets: torch.Tensor) -> torch.Tensor:
        """forward computes the mean reciprocal rank between logits and targets.

        Parameters
        ----------
        logits : torch.Tensor
            Class probability
        targets : torch.Tensor
            Ground truth class indices

        Returns
        -------
        torch.Tensor
            The mean reciprocal rank, a scalar-tensor.
        """

        _, indices = logits.topk(logits.shape[-1], dim=-1)
        ranks = (indices == targets.unsqueeze(-1)).nonzero()[:, -1].float() + 1
        return (1.0 / ranks).mean()

## Dataset and Datamodule

We then define a pytorch dataset and a custom collation function that allows us to dynamically
pad sequences to the longest one in the batch (as opposed to the longest one in the dataset)
as they are loaded during training, this gives us an edge in performance by dramatically reducing the
sparsity of our inputs.

In [68]:
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence


def rnn_collation_fn(batch):

    users = []
    pois = []
    g2 = []
    g3 = []
    g4 = []
    g5 = []
    g6 = []
    
    len_user = []
    len_poi = []
    len_g2 = []
    len_g3 = []
    len_g4 = []
    len_g5 = []
    len_g6 = []

    for user, poi, geo2, geo3, geo4, geo5, geo6 in batch:
        users.append(user)
        pois.append(poi)
        g2.append(geo2)
        g3.append(geo3)
        g4.append(geo4)
        g5.append(geo5)
        g6.append(geo6)
        
        len_user.append(len(users))
        len_poi.append(len(pois))
        len_g2.append(len(g2))
        len_g3.append(len(g3))
        len_g4.append(len(g4))
        len_g5.append(len(g5))
        len_g6.append(len(g6))
        
        
        
    seq = (
        torch.tensor(users, dtype=torch.long),
        pad_sequence(pois, batch_first=True, padding_value=0),
        pad_sequence(g2, batch_first=True, padding_value=0),
        pad_sequence(g3, batch_first=True, padding_value=0),
        pad_sequence(g4, batch_first=True, padding_value=0),
        pad_sequence(g5, batch_first=True, padding_value=0),
        pad_sequence(g6, batch_first=True, padding_value=0),
    )  # build a sequence

    x = (
        seq[0],
        seq[1][:, :-1],
        seq[2][:, :-1],
        seq[3][:, :-1],
        seq[4][:, :-1],
        seq[5][:, :-1],
        seq[6][:, :-1],
    )  # omit the last one for sample

    y = (
        seq[0],
        seq[1][:, 1:],
        seq[2][:, 1:],
        seq[3][:, 1:],
        seq[4][:, 1:],
        seq[5][:, 1:],
        seq[6][:, 1:],
    )  # omit the first one for ground truth
    
    length_pois = torch.tensor(len_poi)
    
    return x, y, length_pois


class CheckinDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data["users"])

    def __getitem__(self, idx):

        x = (
            torch.tensor(encoded_data["users"][idx], dtype=torch.long),
            torch.tensor(encoded_data["pois"][idx], dtype=torch.long),
            torch.tensor(encoded_data["g2"][idx], dtype=torch.long),
            torch.tensor(encoded_data["g3"][idx], dtype=torch.long),
            torch.tensor(encoded_data["g4"][idx], dtype=torch.long),
            torch.tensor(encoded_data["g5"][idx], dtype=torch.long),
            torch.tensor(encoded_data["g6"][idx], dtype=torch.long),
        )

        return x

In [69]:
class CheckinModule(pl.LightningDataModule):
    def __init__(self, encoded_data_train, encoded_data_test, batch_size=32, workers=4):
        """__init__ initializes the CheckinModule.

        Parameters
        ----------
        encoded_data_train : Union[dict, rs.DataFrame]
            The training data.
        encoded_data_test : Union[dict, rs.DataFrame]
            The testing data.
        batch_size : int, optional
            Size of the batches, by default 32
        workers : int, optional
            Number of worker processes, by default 4
        """
        super().__init__()
        self.encoded_data_train = encoded_data_train
        self.encoded_data_test = encoded_data_test
        self.batch_size = batch_size
        self.workers = workers

        assert isinstance(self.encoded_data_train, dict) or isinstance(
            self.encoded_data_train, rs.DataFrame
        ), "encoded_data_train must be a dict or a polars DataFrame"
        assert isinstance(self.encoded_data_test, dict) or isinstance(
            self.encoded_data_test, rs.DataFrame
        ), "encoded_data_test must be a dict or a polars DataFrame"

        assert batch_size > 0, "batch_size must be a positive integer"
        assert workers >= 0, "workers must be a non-negative integer"

    def setup(self, stage=None):

        if (
            isinstance(self.encoded_data_train, dict)
            or isinstance(self.encoded_data_train, rs.DataFrame)
        ) and (
            isinstance(self.encoded_data_test, dict)
            or isinstance(self.encoded_data_test, rs.DataFrame)
        ):
            print("Loading data from dict/dataframe")
            self.train_dataset = CheckinDataset(self.encoded_data_train)
            self.test_dataset = CheckinDataset(self.encoded_data_test)
            print(len(self.encoded_data_train["users"]))
        elif isinstance(self.encoded_data_train, CheckinDataset) and isinstance(
            self.encoded_data_test, CheckinDataset
        ):
            print("Loading data from pre-instantiated datasets")
            self.train_dataset = self.encoded_data_train
            self.test_dataset = self.encoded_data_test
        else:
            raise ValueError("Invalid data type")

    def train_dataloader(self):
        return torch.utils.data.DataLoader(
            self.train_dataset,
            batch_size=self.batch_size,
            shuffle=True,
            num_workers=self.workers,
            collate_fn=rnn_collation_fn,
        )

    def val_dataloader(self):
        return torch.utils.data.DataLoader(
            self.test_dataset,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=self.workers,
            collate_fn=rnn_collation_fn,
        )

    def test_dataloader(self):
        return torch.utils.data.DataLoader(
            self.test_dataset,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=self.workers,
            collate_fn=rnn_collation_fn,
        )

    def save(self, whole_path, train_path, test_path):

        torch.save(self.train_dataset, train_path)
        torch.save(self.test_dataset, test_path)

    @staticmethod  # load without instantiating
    def load(train_path, test_path):

        train_dataset = torch.load(train_path)
        test_dataset = torch.load(test_path)

        return CheckinModule(train_dataset, test_dataset)

## Baseline model: LSTM

In [81]:
@dataclass
class BaselineDimensions:
    nuser: int
    npoi: int
    g2len: int
    g3len: int
    g4len: int
    g5len: int
    g6len: int


# HMT_RN (Hierarchical Multi-Task Recurrent Network)
class HMT_RN(pl.LightningModule):
    def __init__(
        self,
        dimensions: BaselineDimensions,
        embedding_dim,
        lstm_hidden_dim,
        dropout_rate=0.9,
        # 0.9 is a lot, but the paper says so.
    ):
        super(HMT_RN, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = lstm_hidden_dim
        self.dims = dimensions
        

        # Embedding layers one for user, one for poi and one for each G@P
        self.user_embedding = nn.Embedding(
            dimensions.nuser, embedding_dim, padding_idx=0
        )
        self.poi_embedding = nn.Embedding(dimensions.npoi, embedding_dim, padding_idx=0)
        self.g2_embed = nn.Embedding(dimensions.g2len, embedding_dim, padding_idx=0)
        self.g3_embed = nn.Embedding(dimensions.g3len, embedding_dim, padding_idx=0)
        self.g4_embed = nn.Embedding(dimensions.g4len, embedding_dim, padding_idx=0)
        self.g5_embed = nn.Embedding(dimensions.g5len, embedding_dim, padding_idx=0)
        self.g6_embed = nn.Embedding(dimensions.g6len, embedding_dim, padding_idx=0)

        # Dropout layer for embeddings
        self.e_drop = nn.Dropout(p=dropout_rate)
        
        
        # LSTM layer
        self.lstm = nn.LSTM(
            input_size=embedding_dim, hidden_size=lstm_hidden_dim, batch_first=True
        )

        # Linear layers for prediction tasks
        self.linear_poi = nn.Linear(lstm_hidden_dim + embedding_dim, dimensions.npoi)
        self.linear_g2 = nn.Linear(lstm_hidden_dim + embedding_dim, dimensions.g2len)
        self.linear_g3 = nn.Linear(lstm_hidden_dim + embedding_dim, dimensions.g3len)
        self.linear_g4 = nn.Linear(lstm_hidden_dim + embedding_dim, dimensions.g4len)
        self.linear_g5 = nn.Linear(lstm_hidden_dim + embedding_dim, dimensions.g5len)
        self.linear_g6 = nn.Linear(lstm_hidden_dim + embedding_dim, dimensions.g6len)

        self.criterion = nn.CrossEntropyLoss()

        self.top1_argmax = AccuracyAt1()
        self.top1 = AccuracyAtK(1)
        self.top5 = AccuracyAtK(5)
        self.top10 = AccuracyAtK(10)
        self.top20 = AccuracyAtK(20)
        self.mrr = MeanReciprocalRank()

        self.apply(self.init_weights)

    def init_weights(self, w):

        if type(w) == nn.Linear:
            nn.init.kaiming_normal_(w.weight)
            nn.init.constant_(w.bias, 0)
        elif type(w) == nn.LSTM:
            for name, param in w.named_parameters():
                if "bias" in name:
                    nn.init.constant_(param, 0)
                elif "weight" in name:
                    nn.init.kaiming_normal_(param)
        elif type(w) == nn.Embedding:
            nn.init.kaiming_normal_(w.weight)
            nn.init.constant_(w.weight[0], 0)

    def forward(self, batch, lengths):
        """forward passes the batch through the model.

        Parameters
        ----------
        batch : `tuple[torch.Tensor]`
            A tuple of tensors ordered as follows:
            (users, poi, x_geoHash2, x_geoHash3, x_geoHash4, x_geoHash5, x_geoHash6)
        """

        users, poi, x_geoHash2, x_geoHash3, x_geoHash4, x_geoHash5, x_geoHash6 = batch

        B, T = poi.shape

        # make it so  that users are tiled T times
        users = users.repeat(T, 1).T

        e_user = self.e_drop(self.user_embedding(users))
        e_poi = self.e_drop(self.poi_embedding(poi))
        e_gap2 = self.e_drop(self.g2_embed(x_geoHash2))
        e_gap3 = self.e_drop(self.g3_embed(x_geoHash3))
        e_gap4 = self.e_drop(self.g4_embed(x_geoHash4))
        e_gap5 = self.e_drop(self.g5_embed(x_geoHash5))
        e_gap6 = self.e_drop(self.g6_embed(x_geoHash6))
        
        packed_input = nn.utils.rnn.pack_padded_sequence(e_poi, lengths, batch_first=True, enforce_sorted=False)
        pack_out, (h_t, c_t) = self.lstm(packed_input)
        output, _ = nn.utils.rnn.pad_packed_sequence(pack_out, batch_first=True)

        # dense layers
        next_poi = self.linear_poi(torch.cat((output, e_user), dim=2))
        next_g2 = self.linear_g2(torch.cat((output, e_gap2), dim=2))
        next_g3 = self.linear_g3(torch.cat((output, e_gap3), dim=2))
        next_g4 = self.linear_g4(torch.cat((output, e_gap4), dim=2))
        next_g5 = self.linear_g5(torch.cat((output, e_gap5), dim=2))
        next_g6 = self.linear_g6(torch.cat((output, e_gap6), dim=2))

        return next_poi, next_g2, next_g3, next_g4, next_g5, next_g6

    def training_step(self, batch, batch_idx):
        x, y, lenpoi = batch

        (
            poi_pred,
            gap2_pred,
            gap3_pred,
            gap4_pred,
            gap5_pred,
            gap6_pred,
        ) = self(x, lenpoi)

        loss_poi = self.criterion(
            poi_pred.reshape(-1, self.dims.npoi), y[1].reshape(-1)
        )
        loss_gap2 = self.criterion(
            gap2_pred.reshape(-1, self.dims.g2len), y[2].reshape(-1)
        )
        loss_gap3 = self.criterion(
            gap3_pred.reshape(-1, self.dims.g3len), y[3].reshape(-1)
        )
        loss_gap4 = self.criterion(
            gap4_pred.reshape(-1, self.dims.g4len), y[4].reshape(-1)
        )
        loss_gap5 = self.criterion(
            gap5_pred.reshape(-1, self.dims.g5len), y[5].reshape(-1)
        )
        loss_gap6 = self.criterion(
            gap6_pred.reshape(-1, self.dims.g6len), y[6].reshape(-1)
        )

        loss = (
            loss_poi + loss_gap2 + loss_gap3 + loss_gap4 + loss_gap5 + loss_gap6
        ) / 6
        self.log("train/loss", loss)
        self.log("train/loss_gap2", loss_gap2)
        self.log("train/loss_gap3", loss_gap3)
        self.log("train/loss_gap4", loss_gap4)
        self.log("train/loss_gap5", loss_gap5)
        self.log("train/loss_gap6", loss_gap6)
        self.log("train/loss_poi", loss_poi)

        return {"loss": loss}

    def validation_step(self, batch, batch_idx):
        x, y, lenpoi = batch
        (
            poi_pred,
            gap2_pred,
            gap3_pred,
            gap4_pred,
            gap5_pred,
            gap6_pred,
        ) = self(x, lenpoi)

        loss_poi = self.criterion(
            poi_pred.reshape(-1, self.dims.npoi), y[1].reshape(-1)
        )
        loss_gap2 = self.criterion(
            gap2_pred.reshape(-1, self.dims.g2len), y[2].reshape(-1)
        )
        loss_gap3 = self.criterion(
            gap3_pred.reshape(-1, self.dims.g3len), y[3].reshape(-1)
        )
        loss_gap4 = self.criterion(
            gap4_pred.reshape(-1, self.dims.g4len), y[4].reshape(-1)
        )
        loss_gap5 = self.criterion(
            gap5_pred.reshape(-1, self.dims.g5len), y[5].reshape(-1)
        )
        loss_gap6 = self.criterion(
            gap6_pred.reshape(-1, self.dims.g6len), y[6].reshape(-1)
        )

        loss = (
            loss_poi + loss_gap2 + loss_gap3 + loss_gap4 + loss_gap5 + loss_gap6
        ) / 6

        top1_argmax=self.top1_argmax(poi_pred,y[1])
        top1_acc = self.top1(poi_pred,y[1])
        top5_acc = self.top5(poi_pred, y[1])
        top10_acc = self.top10(poi_pred, y[1])
        top20_acc = self.top20(poi_pred, y[1])
        mrr = self.mrr(poi_pred, y[1])

        self.log("val/loss", loss)
        self.log("val/loss_gap2", loss_gap2)
        self.log("val/loss_gap3", loss_gap3)
        self.log("val/loss_gap4", loss_gap4)
        self.log("val/loss_gap5", loss_gap5)
        self.log("val/loss_gap6", loss_gap6)
        self.log("val/loss_poi", loss_poi)

        # log "leaderboard" metrics
        self.log("val/top1_argmax", top1_argmax)
        self.log("val/top1", top1_acc)
        self.log("val/top5", top5_acc)
        self.log("val/top10", top10_acc)
        self.log("val/top20", top20_acc)
        self.log("val/mrr", mrr)

        return loss

    def test_step(self, batch, batch_idx):
        x, y, lenpoi = batch
        (
            poi_pred,
            gap2_pred,
            gap3_pred,
            gap4_pred,
            gap5_pred,
            gap6_pred,
        ) = self(x, lenpoi)

        loss_poi = self.criterion(
            poi_pred.reshape(-1, self.dims.npoi), y[1].reshape(-1)
        )
        loss_gap2 = self.criterion(
            gap2_pred.reshape(-1, self.dims.g2len), y[2].reshape(-1)
        )
        loss_gap3 = self.criterion(
            gap3_pred.reshape(-1, self.dims.g3len), y[3].reshape(-1)
        )
        loss_gap4 = self.criterion(
            gap4_pred.reshape(-1, self.dims.g4len), y[4].reshape(-1)
        )
        loss_gap5 = self.criterion(
            gap5_pred.reshape(-1, self.dims.g5len), y[5].reshape(-1)
        )
        loss_gap6 = self.criterion(
            gap6_pred.reshape(-1, self.dims.g6len), y[6].reshape(-1)
        )

        loss = (
            loss_poi + loss_gap2 + loss_gap3 + loss_gap4 + loss_gap5 + loss_gap6
        ) / 6

        self.log("test/loss", loss)
        self.log("test/loss_gap2", loss_gap2)
        self.log("test/loss_gap3", loss_gap3)
        self.log("test/loss_gap4", loss_gap4)
        self.log("test/loss_gap5", loss_gap5)
        self.log("test/loss_gap6", loss_gap6)
        self.log("test/loss_poi", loss_poi)

        top1_acc = self.top1(poi_pred.reshape(-1, self.dims.npoi), y[1].reshape(-1))
        top5_acc = self.top5(poi_pred.reshape(-1, self.dims.npoi), y[1].reshape(-1))
        top10_acc = self.top10(poi_pred.reshape(-1, self.dims.npoi), y[1].reshape(-1))
        top20_acc = self.top20(poi_pred.reshape(-1, self.dims.npoi), y[1].reshape(-1))
        mrr = self.mrr(poi_pred.reshape(-1, self.dims.npoi), y[1].reshape(-1))

        # log "leaderboard" metrics
        self.log("test/top1", top1_acc)
        self.log("test/top5", top5_acc)
        self.log("test/top10", top10_acc)
        self.log("test/top20", top20_acc)
        self.log("test/mrr", mrr)

        return {"loss": loss}

    def configure_optimizers(self):
        # Define optimizer and scheduler
        optimizer = torch.optim.AdamW(self.parameters(), lr=1e-4, amsgrad=True)
        return optimizer

## Graph Neural Network

In [82]:
# GNN Components


class attn_LSTM(pl.LightningModule):

    def __init__(self, embedding_dim, hidden_dim):
        super(attn_LSTM, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.W = nn.Linear(embedding_dim, 4 * hidden_dim)
        self.U = nn.Linear(hidden_dim, 4 * hidden_dim)

        self.s_W = nn.Linear(embedding_dim, 4 * hidden_dim)
        self.t_W = nn.Linear(embedding_dim, 4 * hidden_dim)

    def forward(self, x, hidden, spatial, temporal, numTimeSteps):
        h_t, c_t = hidden

        previous_h_t = h_t
        previous_c_t = c_t

        allGates_preact = (
            self.W(x) + self.U(previous_h_t) + self.s_W(spatial) + self.t_W(temporal)
        )

        input_g = allGates_preact[:, :, : self.hidden_dim].sigmoid()
        forget_g = allGates_preact[
            :, :, self.hidden_dim : 2 * self.hidden_dim
        ].sigmoid()
        output_g = allGates_preact[
            :, :, 2 * self.hidden_dim : 3 * self.hidden_dim
        ].sigmoid()
        c_t_g = allGates_preact[:, :, 3 * self.hidden_dim :].tanh()

        c_t = forget_g * previous_c_t + input_g * c_t_g
        h_t = output_g * c_t.tanh()

        batchSize = x.shape[0]
        h_t = h_t.view(batchSize, numTimeSteps, self.hidden_dim)
        c_t = c_t.view(batchSize, numTimeSteps, self.hidden_dim)

        return h_t, c_t


def get_neighbours(adj_matrix, poi):
    neigh_indices_list = []
    max_length = 0

    for batch_poi in poi:
        batch_indices = []
        for single_poi in batch_poi:
            poi_row = adj_matrix[single_poi]
            neigh_indices = torch.where(poi_row == 1)[0]
            batch_indices.append(neigh_indices)
            max_length = max(max_length, len(neigh_indices))

        neigh_indices_list.append(batch_indices)

    padded_neigh_indices_list = []
    for batch_indices in neigh_indices_list:
        padded_batch_indices = pad_sequence(
            batch_indices, batch_first=True, padding_value=0
        )
        padded_neigh_indices_list.append(padded_batch_indices)

    padded_tensor = torch.stack(padded_neigh_indices_list)

    return padded_tensor


class GRNSelfAttention(torch.nn.Module):

    def __init__(self, hidden_dim, n_heads):

        super(GRNSelfAttention, self).__init__()

        self.hidden_dim = hidden_dim
        self.n_heads = n_heads

        self.Wp = nn.Linear(hidden_dim, hidden_dim)  # embeddings to pre-concat
        self.Wa = nn.Linear(2 * hidden_dim, hidden_dim)  # concatenation to pre-softmax

        # total size = 3 * (hidden_dim) ** 2, quadratic in embedding size

    def forward(self, poi, neighbors):
        """forward

        Parameters
        ----------
        poi: torch.Tensor
            A batched tensor of embedded POI vectors, (B x H) where H is the
            embedding dimension
        neighbors: torch.Tensor
            A batched tensor of sequences of embedded POI vectors that are extracted
            from an adjacency matrix (temporal or spatial neighbors of POI),
            (B x N x H), where N is the number of neighbours of POI, B is the
            batch size, H is the embedding dimension, and must be the same as POI

        Returns
        -------
        tuple[torch.Tensor, torch.Tensor]
          A tuple containing the self-attention weighted hadamard product of neighbour activations
          in the first index, the attention weights in the second index.
        """
        # assert len(poi.shape) == 2, f"POI tensor must be 2D, got {poi.shape} instead"
        assert (
            len(neighbors.shape) == 3
        ), f"Neighbour tensor must be 3D, got {neighbors.shape} instead"

        B, N, H = neighbors.shape

        h_poi = self.Wp(poi)
        h_n = self.Wp(neighbors)
        h_cat = torch.cat([h_poi.expand(B, N, -1), h_n], dim=2)
        h_att = F.leaky_relu(self.Wa(h_cat))

        alpha = torch.nn.functional.softmax(h_att, dim=1)

        p = torch.sum(alpha * h_n, dim=1)
        return p, alpha

In [83]:
# GRN (Graph Recurrent Network)
class GRN(pl.LightningModule):

    def __init__(
        self,
        dims: BaselineDimensions,
        spatial_graph,
        temporal_graph,
        hidden_dim,
        n_heads,
        dropout_rate=0.9,
        device="cpu",
    ):
        super(GRN, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_heads = n_heads
        self.dims = dims

        self.spatial_graph = spatial_graph.to(device)
        self.temporal_graph = temporal_graph.to(device)

        self.spatial_attn = GRNSelfAttention(hidden_dim, n_heads)
        self.temporal_attn = GRNSelfAttention(hidden_dim, n_heads)

        self.lstm = attn_LSTM(hidden_dim, hidden_dim)

        self.dropout = nn.Dropout(dropout_rate)

        self.user_embedding = nn.Embedding(dims.nuser, hidden_dim, padding_idx=0)
        self.poi_embedding = nn.Embedding(dims.npoi, hidden_dim, padding_idx=0)
        self.g2_embed = nn.Embedding(dims.g2len, hidden_dim, padding_idx=0)
        self.g3_embed = nn.Embedding(dims.g3len, hidden_dim, padding_idx=0)
        self.g4_embed = nn.Embedding(dims.g4len, hidden_dim, padding_idx=0)
        self.g5_embed = nn.Embedding(dims.g5len, hidden_dim, padding_idx=0)
        self.g6_embed = nn.Embedding(dims.g6len, hidden_dim, padding_idx=0)

        self.linear_poi = nn.Linear(2 * hidden_dim, dims.npoi)
        self.linear_g2 = nn.Linear(2 * hidden_dim, dims.g2len)
        self.linear_g3 = nn.Linear(2 * hidden_dim, dims.g3len)
        self.linear_g4 = nn.Linear(2 * hidden_dim, dims.g4len)
        self.linear_g5 = nn.Linear(2 * hidden_dim, dims.g5len)
        self.linear_g6 = nn.Linear(2 * hidden_dim, dims.g6len)

        self.top1_argmax=AccuracyAt1()
        self.top1 = AccuracyAtK(1)
        self.top5 = AccuracyAtK(5)
        self.top10 = AccuracyAtK(10)
        self.top20 = AccuracyAtK(20)
        self.mrr = MeanReciprocalRank()

        # extract indices from one-hot neighbor list
        self.iota = torch.arange(self.dims.npoi, requires_grad=False, device=device)

        self.criterion = nn.CrossEntropyLoss()

        self.apply(self.init_weights)

    def init_weights(self, w):
        if type(w) == nn.Linear:
            nn.init.kaiming_normal_(w.weight)
            nn.init.constant_(w.bias, 0)
        elif type(w) == nn.LSTM:
            for name, param in w.named_parameters():
                if "bias" in name:
                    nn.init.constant_(param, 0)
                elif "weight" in name:
                    nn.init.kaiming_normal_(param)
        elif type(w) == nn.Embedding:
            nn.init.kaiming_normal_(w.weight)
            nn.init.constant_(w.weight[0], 0)

    def forward(self, x, lengths):

        users, poi, x_geoHash2, x_geoHash3, x_geoHash4, x_geoHash5, x_geoHash6 = x

        B, T = poi.shape

        users = users.repeat(T, 1).T

        neighbors_spatial = self.spatial_graph[poi]
        neighbors_temporal = self.temporal_graph[poi]

        e_user = self.dropout(self.user_embedding(users))
        e_poi = self.dropout(self.poi_embedding(poi))
        e_gap2 = self.dropout(self.g2_embed(x_geoHash2))
        e_gap3 = self.dropout(self.g3_embed(x_geoHash3))
        e_gap4 = self.dropout(self.g4_embed(x_geoHash4))
        e_gap5 = self.dropout(self.g5_embed(x_geoHash5))
        e_gap6 = self.dropout(self.g6_embed(x_geoHash6))

        spatial_atts = torch.empty((B, T, self.hidden_dim), device=self.device)
        temporal_atts = torch.empty((B, T, self.hidden_dim), device=self.device)

        for b in range(B):
            for t in range(T):

                spatial_neigh = neighbors_spatial[b, t] * self.iota
                temporal_neigh = neighbors_temporal[b, t] * self.iota

                spatial_neigh = spatial_neigh[spatial_neigh != 0]
                temporal_neigh = temporal_neigh[temporal_neigh != 0]

                spatial_neigh = spatial_neigh.unsqueeze(0)
                temporal_neigh = temporal_neigh.unsqueeze(0)

                e_spatial = self.dropout(self.poi_embedding(spatial_neigh))
                e_temporal = self.dropout(self.poi_embedding(temporal_neigh))

                curr_poi = e_poi[b, t].unsqueeze(0)

                spatial_p, _ = self.spatial_attn(curr_poi, e_spatial)
                temporal_p, _ = self.temporal_attn(curr_poi, e_temporal)

                # we are not using the batch dimension, so we squeeze it
                spatial_atts[b, t] = spatial_p.squeeze()
                temporal_atts[b, t] = temporal_p.squeeze()

        # zero-init LSTM states
        h_t = torch.zeros(B, T, self.hidden_dim, device=self.device)
        c_t = torch.zeros(B, T, self.hidden_dim, device=self.device)
        
        packed_input = nn.utils.rnn.pack_padded_sequence(e_poi, lengths, batch_first=True, enforce_sorted=False)
        pack_out, (h_t, c_t) = self.lstm(packed_input, (h_t, c_t), spatial_atts, temporal_atts, T)
        output, _ = nn.utils.rnn.pad_packed_sequence(pack_out, batch_first=True)

        # Note:the prediction of the poi depends on the embedding o the user
        next_poi = self.linear_poi(torch.cat((output, e_user), dim=2))
        next_g2 = self.linear_g2(torch.cat((output, e_gap2), dim=2))
        next_g3 = self.linear_g3(torch.cat((output, e_gap3), dim=2))
        next_g4 = self.linear_g4(torch.cat((output, e_gap4), dim=2))
        next_g5 = self.linear_g5(torch.cat((output, e_gap5), dim=2))
        next_g6 = self.linear_g6(torch.cat((output, e_gap6), dim=2))

        return next_poi, next_g2, next_g3, next_g4, next_g5, next_g6

    def training_step(self, batch, batch_idx):
        x, y, lenpoi = batch
        (
            poi_pred,
            gap2_pred,
            gap3_pred,
            gap4_pred,
            gap5_pred,
            gap6_pred,
        ) = self(x, lenpoi)

        loss_poi = self.criterion(
            poi_pred.reshape(-1, self.dims.npoi), y[1].reshape(-1)
        )
        loss_gap2 = self.criterion(
            gap2_pred.reshape(-1, self.dims.g2len), y[2].reshape(-1)
        )
        loss_gap3 = self.criterion(
            gap3_pred.reshape(-1, self.dims.g3len), y[3].reshape(-1)
        )
        loss_gap4 = self.criterion(
            gap4_pred.reshape(-1, self.dims.g4len), y[4].reshape(-1)
        )
        loss_gap5 = self.criterion(
            gap5_pred.reshape(-1, self.dims.g5len), y[5].reshape(-1)
        )
        loss_gap6 = self.criterion(
            gap6_pred.reshape(-1, self.dims.g6len), y[6].reshape(-1)
        )

        loss = (
            loss_poi + loss_gap2 + loss_gap3 + loss_gap4 + loss_gap5 + loss_gap6
        ) / 6
        self.log("train/loss", loss)
        self.log("train/loss_gap2", loss_gap2)
        self.log("train/loss_gap3", loss_gap3)
        self.log("train/loss_gap4", loss_gap4)
        self.log("train/loss_gap5", loss_gap5)
        self.log("train/loss_gap6", loss_gap6)
        self.log("train/loss_poi", loss_poi)

        return {"loss": loss}

    def validation_step(self, batch, batch_idx):
        x, y, lenpoi = batch
        (
            poi_pred,
            gap2_pred,
            gap3_pred,
            gap4_pred,
            gap5_pred,
            gap6_pred,
        ) = self(x, lenpoi)

        loss_poi = self.criterion(
            poi_pred.reshape(-1, self.dims.npoi), y[1].reshape(-1)
        )
        loss_gap2 = self.criterion(
            gap2_pred.reshape(-1, self.dims.g2len), y[2].reshape(-1)
        )
        loss_gap3 = self.criterion(
            gap3_pred.reshape(-1, self.dims.g3len), y[3].reshape(-1)
        )
        loss_gap4 = self.criterion(
            gap4_pred.reshape(-1, self.dims.g4len), y[4].reshape(-1)
        )
        loss_gap5 = self.criterion(
            gap5_pred.reshape(-1, self.dims.g5len), y[5].reshape(-1)
        )
        loss_gap6 = self.criterion(
            gap6_pred.reshape(-1, self.dims.g6len), y[6].reshape(-1)
        )

        loss = (
            loss_poi + loss_gap2 + loss_gap3 + loss_gap4 + loss_gap5 + loss_gap6
        ) / 6

        self.log("val/loss", loss)
        self.log("val/loss_gap2", loss_gap2)
        self.log("val/loss_gap3", loss_gap3)
        self.log("val/loss_gap4", loss_gap4)
        self.log("val/loss_gap5", loss_gap5)
        self.log("val/loss_gap6", loss_gap6)
        self.log("val/loss_poi", loss_poi)

        top1_argmax=self.top1_argmax(poi_pred.reshape(-1, self.dims.npoi), y[1].reshape(-1))
        top1_acc =self.top1(poi_pred.reshape(-1, self.dims.npoi), y[1].reshape(-1))
        top5_acc = self.top5(poi_pred.reshape(-1, self.dims.npoi), y[1].reshape(-1))
        top10_acc = self.top10(poi_pred.reshape(-1, self.dims.npoi), y[1].reshape(-1))
        top20_acc = self.top20(poi_pred.reshape(-1, self.dims.npoi), y[1].reshape(-1))
        mrr = self.mrr(poi_pred.reshape(-1, self.dims.npoi), y[1].reshape(-1))

        # log "leaderboard" metrics
        self.log("val/top1_argmax", top1_argmax)
        self.log("val/top1", top1_acc)
        self.log("val/top5", top5_acc)
        self.log("val/top10", top10_acc)
        self.log("val/top20", top20_acc)
        self.log("val/mrr", mrr)

        return loss

    def test_step(self, batch, batch_idx):
        x, y, lenpoi = batch
        (
            poi_pred,
            gap2_pred,
            gap3_pred,
            gap4_pred,
            gap5_pred,
            gap6_pred,
        ) = self(x)

        loss_poi = self.criterion(
            poi_pred.reshape(-1, self.dims.npoi), y[1].reshape(-1)
        )
        loss_gap2 = self.criterion(
            gap2_pred.reshape(-1, self.dims.g2len), y[2].reshape(-1)
        )
        loss_gap3 = self.criterion(
            gap3_pred.reshape(-1, self.dims.g3len), y[3].reshape(-1)
        )
        loss_gap4 = self.criterion(
            gap4_pred.reshape(-1, self.dims.g4len), y[4].reshape(-1)
        )
        loss_gap5 = self.criterion(
            gap5_pred.reshape(-1, self.dims.g5len), y[5].reshape(-1)
        )
        loss_gap6 = self.criterion(
            gap6_pred.reshape(-1, self.dims.g6len), y[6].reshape(-1)
        )

        loss = (
            loss_poi + loss_gap2 + loss_gap3 + loss_gap4 + loss_gap5 + loss_gap6
        ) / 6

        self.log("test/loss", loss)
        self.log("test/loss_gap2", loss_gap2)
        self.log("test/loss_gap3", loss_gap3)
        self.log("test/loss_gap4", loss_gap4)
        self.log("test/loss_gap5", loss_gap5)
        self.log("test/loss_gap6", loss_gap6)
        self.log("test/loss_poi", loss_poi)

        top1_acc = self.top1(poi_pred, y[1])
        top5_acc = self.top5(poi_pred, y[1])
        top10_acc = self.top10(poi_pred, y[1])
        top20_acc = self.top20(poi_pred, y[1])
        mrr = self.mrr(poi_pred, y[1])

        # log "leaderboard" metrics
        self.log("test/top1", top1_acc)
        self.log("test/top5", top5_acc)
        self.log("test/top10", top10_acc)
        self.log("test/top20", top20_acc)
        self.log("test/mrr", mrr)

        return {"loss": loss}

    def configure_optimizers(self):
        # Define optimizer and scheduler
        optimizer = torch.optim.AdamW(self.parameters(), lr=1e-4, amsgrad=True)
        return optimizer

## Training Loops

In [84]:
n_users = encoder_dict["users"].classes_.shape[0]
n_pois = encoder_dict["pois"].classes_.shape[0]
n_g2 = encoder_dict["g2"].classes_.shape[0]
n_g3 = encoder_dict["g3"].classes_.shape[0]
n_g4 = encoder_dict["g4"].classes_.shape[0]
n_g5 = encoder_dict["g5"].classes_.shape[0]
n_g6 = encoder_dict["g6"].classes_.shape[0]


# account for the padding token
dims = BaselineDimensions(
    n_users + 1, n_pois + 1, n_g2 + 1, n_g3 + 1, n_g4 + 1, n_g5 + 1, n_g6 + 1
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [85]:
from lightning.pytorch.loggers import WandbLogger
from lightning.pytorch import Trainer

TRAIN_BASELINE = True

wandb.finish()
torch.cuda.empty_cache()
# cargo-cult like stuff that is supposed to make you faster
torch.set_float32_matmul_precision("medium")
torch.backends.cudnn.benchmark = True

ds = CheckinModule(encoded_data_train, encoded_data_test, batch_size=32, workers=4)

wandb.init(project="trovailpoi")

classifier_baseline = HMT_RN(dims, embedding_dim=1024, lstm_hidden_dim=1024)
wandb_logger = WandbLogger(project="trovailpoi")
trainer = Trainer(
    max_epochs=10,
    accelerator="auto",
    devices=[0],
    log_every_n_steps=10,
    logger=wandb_logger,
    strategy="auto",
    callbacks=[
        torchpl.callbacks.LearningRateMonitor(logging_interval="step"),
        torchpl.callbacks.ModelCheckpoint(
            monitor="val/loss",
            mode="min",
            save_top_k=1,
            save_last=True,
            filename="best_model",
        ),
        torchpl.callbacks.EarlyStopping(
            monitor="val/loss", patience=3, min_delta=0.0005, mode="min"
        ),
    ],
)

if TRAIN_BASELINE:
    trainer.fit(model=classifier_baseline, datamodule=ds)
wandb.finish()

INFO: GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
INFO: 
   | Name           | Type               | Params
-------------------------------------------------------
0  | user_embedding | Embedding          | 20.3 M
1  | poi_embedding  | Embedding          | 4.6 M 
2  | g2_embed       | Embedding          | 114 K 
3  | g3_embed       | Embedding          | 287 K 
4  | g4_embed       | Embedding          | 561 K 
5  | g5_embed       | Embedding          | 1.4 M 
6  | g6_embed       | Embedding          | 2.8 M 
7  | e_drop         | Dropout            | 0     
8  | lstm           | LSTM               | 8.4 M 
9  | linear_poi     | Linear             | 9.1 M 
10 | linear_g2      | Linear             | 229 K 
11 | linear_g3      | Linear             | 575 K 
12 | linear_g4      | Linear             | 1.1 M 
13 | linear

Loading data from dict/dataframe
12060


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

RuntimeError: CUDA error: an illegal memory access was encountered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
TRAIN_GNN = True

batch_size = 60
wandb.finish()
torch.cuda.empty_cache()
# cargo-cult like stuff that is supposed to make you faster
torch.set_float32_matmul_precision("medium")
torch.backends.cudnn.benchmark = True

wandb.init(project="trovailpoi")

classifier_gnn = GRN(
    dims,
    spatial_graph,
    temporal_graph,
    hidden_dim=1024,
    n_heads=1,
    dropout_rate=0.9,
    device=device,
)
wandb_logger = WandbLogger(project="trovailpoi")
trainer = Trainer(
    max_epochs=40,
    accelerator="auto",
    devices=[0],
    log_every_n_steps=10,
    logger=wandb_logger,
    strategy="auto",
    callbacks=[
        torchpl.callbacks.LearningRateMonitor(logging_interval="step"),
        torchpl.callbacks.ModelCheckpoint(
            monitor="val/loss",
            mode="min",
            save_top_k=1,
            save_last=True,
            filename="best_model",
        ),
        torchpl.callbacks.EarlyStopping(
            monitor="val/loss", patience=3, min_delta=0.0005, mode="min"
        ),
    ],
)

if TRAIN_GNN:
    trainer.fit(model=classifier_gnn, datamodule=ds)
wandb.finish()

In [90]:
checkpoint_path = "/content/trovailpoi/dnpotgxc/checkpoints/best_model.ckpt"
# Load the trained model from the checkpoint
trained_model = HMT_RN.load_from_checkpoint(
    checkpoint_path,
    dimensions=dims,
    embedding_dim=1024,  # Example embedding dimension
    lstm_hidden_dim=1024,  # Example LSTM hidden dimension
    dropout_rate=0.9,  # Example dropout rate
)

# Create a test dataloader
# Assuming you have a method `test_dataloader` in your data module
test_loader = ds.test_dataloader()  # Replace `ds` with your actual data module instance

# Instantiate the trainer
trainer = Trainer(accelerator="auto", devices=[0])

# Test the model
results = trainer.test(trained_model, test_loader)

# Print the test results
print(results)

FileNotFoundError: [Errno 2] No such file or directory: '/content/trovailpoi/dnpotgxc/checkpoints/best_model.ckpt'

## Scrapbook for Experimentation

Ignore all code below, it's just for quick prototyping